# Notebook to "try" reproducing certification

This notebook intends to provide pieces to illustrate how to reproduce results from the "Mémoire de Certfication". However there are some limitations.

Some of the data used in the thesis are not available in the dataset and does not allow to have the exact dataset used during the certification. Therefore the results will be different.

Therefore, we will try to have a similar analysis but we will not be able to end-up with similar results.

### Data loading

We will be reusing the data that we clean during the morning of the datacamp.

Note that from the original dataset, we created three new columns:

- the age of the policy at the time of claim
- the delay of the claim
- the age of the client at the time of claim

Those columns were stored as `timedelta` (cf. [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html)) expressed in days. We reuse this format and express these quantity in days to have a numerical representation that does not require complicated processing later on.

In [ ]:
import pandas as pd

df = pd.read_csv(
    "cleaner_dataframe.csv",
    parse_dates=[
        "Claim Incident date",
        "FE_Declaration_date",
        "Initial coverage date",
        "First claim decision date",
        "Last claim decisión date",
        "Policy Holder date of birth",
        "Age policy at claim",
    ],
    index_col=0,
)
df["Age policy at claim"] = pd.to_timedelta(df["Age policy at claim"]).dt.days
df["Delay declaration"] = pd.to_timedelta(df["Delay declaration"]).dt.days
df["Age client at claim"] = pd.to_timedelta(df["Age client at claim"]).dt.days
df

We can check that opening the CSV file when as we expected.

In [ ]:
df.info()

As we can see, dates have been parsed properly and our three new columns are stored with integral data types.

Now, we will split the dataset into a vector containing whether or not a claim was refused and a matrix containing the features. For this matter, we will store the name of the features and the name of the target.

Note that we exclude the date from the features used because we created the last three columns using them.

In [ ]:
feature_names = [
#     "Risk code",
    "Insured amount",
    "Initial_Instalment_Amount",
    "Age at signature",
    "Sexo",
    "Age policy at claim",
    "Delay declaration",
    "Age client at claim",
]
target_names = "Refusal_Flag"
X, y = df[feature_names], df[target_names]

In [ ]:
X.info()

Thus, we have 7 features to learn from.

### Aside note regarding categorical features

As we saw during the lecture, categorical features need particular attention: these features are necessarly numerical values while machine learning algorithm only deal with such numerical values. We therefore needs to encode those values to a numerical representation.

However, this representation needs to be chose with care. Indeed, it will depend on the type of predictive model used later on.

Here, we recall the two main type of categorical encoders used. We also recall with which model one wants to use them.

First let's define in a Python list, which columns can be considered categorical:

In [ ]:
categorical_columns = [
#     "Risk code",
    "Sexo",
]

Indeed, we have two categorical features. While it is obvious that `"Sexo"` is such a categorical features, it is a less obvious for `"Risk code"` since it is represented already by some integral values. However, the name indicate us that this is a "code" and therefore composed of discrete values.

We should therefore include this column in our processing.

In [ ]:
X_categorical = X[categorical_columns]

#### Ordinal encoding

The simplest encoding that we can use is to replace each category by an integer. We will therefore have categories represented by values between `[0, n_categories - 1]`. We will illustrate below such encoding:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_categorical)
X_train_encoded

The output of scikit-learn will be a NumPy array. We can create a pandas `DataFrame` using the `get_feature_names_out` method to generate the column names. This method allows to track complex column transformation in scikit-learn `Pipeline` as well.

In [ ]:
X_train_encoded = pd.DataFrame(
    X_train_encoded, columns=encoder.get_feature_names_out()
)
X_train_encoded

We observe that each categories is therefore encoded with an integer. We could check the correspondence by looking at the fitted attributes of the encoder.

In [ ]:
for col_name, col_categories in zip(
    encoder.get_feature_names_out(),
    encoder.categories_
):
    print(f"For feature named {col_name!r}")
    print()
    for code, cat in enumerate(col_categories):
        print(f"\tCategory {cat!r} mapped to {code}")
    print()

This type of encoder is imposing an ordering and it is not always adequate for linear model in which we don't want to impose such modeling. It is therefore the encoder of choice used in tree-based models.

#### One-hot encoding

When using a linear model, we will be "usually" interested of having an independence between categories. Therefore, with such models, `OneHotEncoder` is an interesting encoder. We will reproduce the previous experiment to show what this encoder does.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
X_train_encoded = encoder.fit_transform(X_categorical)
X_train_encoded

In [ ]:
X_train_encoded = pd.DataFrame(
    X_train_encoded.A, columns=encoder.get_feature_names_out()
)
X_train_encoded

We see that each category becomes a column and that an indicator 0/1 is used to indicate whether or not we deal with the category for this sample.

### Creating basic predictive models

Now, we can design some predictive model. Here, we will develop 3 predictive models:

- a dummy classifier that is not going to use `X` to make any prediction. It is not an interesting model but it provides a kind of baseline.
- a linear model with an adequate preprocessing. Indeed, the preprocessing has to contain a scaler and a one-hot encoder.
- a gradient boosting decision trees model. This model should use an ordinal encoder in its preproceesing stage.

Let's define the three models.

#### Dummy classifier

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_model = DummyClassifier()

#### Linear model

In [ ]:
# define the name of the numerical columns (the ones that are not categorical)
numerical_columns = X.columns.difference(categorical_columns).tolist()
numerical_columns

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

preprocessor = ColumnTransformer(transformers=[
    ("categorical_processor", OneHotEncoder(drop="if_binary"), categorical_columns),
    ("numerical_processor", StandardScaler(), numerical_columns),
])
linear_model = Pipeline(steps=[
    ("preprocessor", preprocessor), ("classifier", LogisticRegression())
])
linear_model

#### Tree-based model

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

preprocessor = ColumnTransformer(transformers=[
        ("categorical_processor", OrdinalEncoder(), categorical_columns),
    ],
    remainder="passthrough",
)
hgbdt_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=0)),
])
hgbdt_model

### Evaluating these models with cross-validation

Now that we define these models, we have to evaluate them. For this purpose, we need to use a metric. However, our problem is imbalanced:

In [ ]:
y.value_counts(normalize=True)

Indeed, the proportion of `"No"` and `"Yes"` is not 50-50. We therefore need to select a proper statistical metric. Here, we can use the ROC-AUC.

We also need to choose a cross-validation strategy. A stratified k-fold cross-validation is a good choice in this case to be sure that the ratio of the classes is preserved in each of the splits. In addition, we will shuffle the data. I will come back on this point.

Let's evaluate the dummy classifier.

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold

cv = StratifiedKFold(n_splits=30, shuffle=True)
cv_results = cross_validate(
    dummy_model, X, y,
    return_train_score=True,
    scoring=["roc_auc"],
    n_jobs=-1,
    cv=cv,
)
cv_results = pd.DataFrame(cv_results)
cv_results = cv_results[["train_roc_auc", "test_roc_auc"]]
print(
    f"""Dummy classifier results in terms of ROC-AUC:

    Training: {cv_results['train_roc_auc'].mean():.3f} +/- {cv_results['train_roc_auc'].std():.3f}
    Testing: {cv_results['test_roc_auc'].mean():.3f} +/- {cv_results['test_roc_auc'].std():.3f}
    """
)

It is not a surprised that this model outputs an average AUC of 0.5 on both training and testing. Let's check if a linear model provides better statistics.

In [ ]:
cv_results = cross_validate(
    linear_model, X, y,
    return_train_score=True,
    scoring=["roc_auc"],
    n_jobs=-1,
    cv=cv,
)
cv_results = pd.DataFrame(cv_results)
cv_results = cv_results[["train_roc_auc", "test_roc_auc"]]
print(
    f"""Linear model results in terms of ROC-AUC:

    Training: {cv_results['train_roc_auc'].mean():.3f} +/- {cv_results['train_roc_auc'].std():.3f}
    Testing: {cv_results['test_roc_auc'].mean():.3f} +/- {cv_results['test_roc_auc'].std():.3f}
    """
)

It seems that this is a bit better. We can also see that the model does not overfit since the results are similar on the training and testing set. Let's check the last model type of model:

In [ ]:
cv_results = cross_validate(
    hgbdt_model, X, y,
    return_train_score=True,
    scoring=["roc_auc"],
    n_jobs=-1,
    cv=cv,
)
cv_results = pd.DataFrame(cv_results)
print(
    f"""HGBDT model results in terms of ROC-AUC:

    Training: {cv_results['train_roc_auc'].mean():.3f} +/- {cv_results['train_roc_auc'].std():.3f}
    Testing: {cv_results['test_roc_auc'].mean():.3f} +/- {cv_results['test_roc_auc'].std():.3f}
    """
)

We observe that this model is better than the previous one. we can see that there is a small overfitting since the model is performing worse on the testing set than on the training set.

### Reproducing ROC curve

In the thesis, the structure was broken to report the results because a single train-test split was used with shuffling. We can reproduce the ROC curve using such approach:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=0
)

In [ ]:
dummy_model.fit(X_train, y_train)
linear_model.fit(X_train, y_train)
_ = hgbdt_model.fit(X_train, y_train)

Now we can use the plotting tool from scikit-learn to plot the ROC curve

In [ ]:
from sklearn.metrics import RocCurveDisplay

disp = RocCurveDisplay.from_estimator(
    dummy_model, X_test, y_test, name="Dummy classifier", linestyle="--"
)
RocCurveDisplay.from_estimator(
    linear_model, X_test, y_test, name="Logistic regression", ax=disp.ax_
)
RocCurveDisplay.from_estimator(
    hgbdt_model, X_test, y_test, name="HGBDT classifier", ax=disp.ax_
)
_ = disp.ax_.set_title(
    "Comparison of the ROC curve of the different models"
)

We more or less obtain the results show in the tesis.

### Permutation importance

Here, we show how to plot the feature importances by using the permutation approach.

In [ ]:
from sklearn.inspection import permutation_importance

importances = permutation_importance(
    hgbdt_model, X_test, y_test, scoring="roc_auc"
)
importances = pd.DataFrame(
    importances.importances.T,
    columns=hgbdt_model[:-1].get_feature_names_out(),
)

In [ ]:
ax = importances.plot.box(vert=False, whis=10)
_ = ax.set_xlabel("Decrease in terms of ROC AUC")

### Conclusion

The thesis is showing the "Economic loss". We cannot construct this loss because we are missing some information. However, it comes back to assign a cost to a prediction depending if we do a mistake or not.

In the thesis, the cost depend on some external values that we don't have access. If we would have these access, we could there fore compute the cost, for all the possible threshold of used for the ROC curve and have a business oriented metric which is more maningful than the ROC curve.